# Pydantic fundamentals

## Start without pydantic

In [1]:
class Person:
    def __init__(self,name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name="Henrik", age=39, gender="M")
person1

In [3]:
person1.name, person1.age

('Henrik', 39)

In [4]:
person2= Person(name=3213+3213, gender=True, age="minus ten")
person2

## validation of out Person class

In [6]:
class Person:
    def __init__(self,name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

try:
    Person(name=3.1415 , age=39, gender="M")
except TypeError as err:
    print(err)

name must be of type str not <class 'float'>


In [10]:
class Person:
    def __init__(self,name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        
        if not isinstance(age, int):
            raise TypeError(f"age must be of type int not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self.age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"

try:
   person3 = Person(name="Ragnar" , age=-39, gender="M")
except ValueError as err:
    print(err)

person3

age must be between 0 and 124 not -39


Person(Ragnar, M, -39)

In [12]:
person4 = Person(name="Bo" , age=2, gender="M")
person4

Person(Bo, M, 2)

In [13]:
person4.age = -2
person4

Person(Bo, M, -2)

In [15]:
class Person:
    def __init__(self,name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"age must be of type int not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self._age = age

    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"
    
person5 = Person(name="August" , age=5, gender="M")
person5

Person(August, M, 5)

In [17]:
try:
    person5.age = -5
except ValueError as err:
    print(err)

age must be between 0 and 124 not -5


In [18]:
try:
    person5.age = -5
except ValueError as err:
    print(err)

age must be between 0 and 124 not -5


# Validate using pydantic

In [19]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person6 = Person(name="Karin", gender="F", age= 45)
person6

Person(name='Karin', gender='F', age=45)

In [20]:
person6.age

45

In [22]:
person6.age=36
person6

Person(name='Karin', gender='F', age=36)

In [24]:
person6.age = "thirty"
person6

Person(name='Karin', gender='F', age='thirty')

In [25]:
Person(name=5346531, gender="F", age= 45)

ValidationError: 1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=5346531, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [26]:
from pydantic import ValidationError

try:
    Person(name=456321, gender="F", age= 45)
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=456321, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [27]:
Person(name="456321", gender="F", age= "45")

Person(name='456321', gender='F', age=45)

In [31]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(validate_assignment=True)

person7 = Person(name="Eva", gender="F", age="72")

try:
    person7.age = "thirty"
except ValidationError as err:
    print(err)

person7.age = 10
person7

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='Eva', gender='F', age=10)

In [32]:
try:
    Person(name=3.1415, gender=2.714, age=-4)
except ValidationError as err:
    print(err)
    

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.1415, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.714, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [34]:
try:
    Person(name="Mats", gender="M", age=-4)
except ValidationError as err:
    print(err)

## Add age validation


In [38]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, lt = 125)

    model_config = ConfigDict(validate_assignment=True)


In [39]:
try:
    Person(name="Mats", gender="feM", age=-4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='feM', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization and dederialization

In [ ]:
# Serialization
person7.model_dump()

{'name': 'Eva', 'gender': 'F', 'age': 10}

In [42]:
import json
with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)

In [47]:
with open("person.json", "r") as file:
    json_data = file.read()

print(repr(json_data))

# deserialization
Person.model_validate_json(json_data)

'{"name": "Eva", "gender": "F", "age": 10}'


Person(name='Eva', gender='F', age=10)